## Do not edit, just constants and useful function definition 

In [1]:

import oommfc as oc
import discretisedfield as df
import micromagneticmodel as mm
import micromagneticdata as md
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import matplotlib.colors as colors

# Conversion factors
Oe = 1000/(4*np.pi)     # conversion Oe->A/m 79.577471/ 1 mT->10 Oe

# magnetic parametes
alpha_YIG = 1.75e-4    # Gilbert damping (-)

Ms_YIG = 140.7e3           # Saturation magnetisation (A/m). 

A_YIG = 4.22e-12           # Exchange stiffness (J/m)

l_ex = (2*A_YIG/(mm.consts.mu0*Ms_YIG**2)) # Exchange length (m)
# Geometry [m]
l = 60e-6
w = 500e-9
t = 50e-9


# self absorbing boundary condition
band = 1e-6

sourceWidth=600e-9 #cercare bene la misura sul righello
sourcePos=-l/4 #almeno iniettiamo le waves nel bulk, possiamo modificare


# Mesh cell [m]
cx = 10e-9   #50e-9
cy = w  #50e-9
cz = t

cell = (cx, cy, cz)

# Zeeman field
Hx = 0 * Oe # 550 Oe = 55mT as in the paper
Hy = 0
Hz = 0
H_DC = (Hx, Hy, Hz)

transverseFieldSize=0.5e-6

# Amplitude of RF exiting field
H_RF_val = 10 * Oe

# Initial magnetization
m0 = (1, 0, 0)   # Initial reduced magnetization


## Edit from here on

In [2]:
f_MAX = 4e9
f_MAX_list=[3.2e9, 3.5e9, 3.8e9, 4e9]

Hy_list = [0, 50, 250, 550, 1100, 2200]
Hy_list = [Hy * Oe for Hy in Hy_list]

T = 100e-9

In [3]:
Ty1=[]
Ty2=[]
Tz1=[]
Tz2=[]

def getTramissions(baseline):
    data=md.Data(sysName) #this contains all the drives up to now. [-1] means the last drive. Check the folder "Py_disk_FMR"
    array=data[-1].to_xarray()
    mz1_left=array[:, int((3/8*l)/cx), int(w/2/cy), 0, 2]
    my1_left=array[:, int((3/8*l)/cx), int(w/2/cy), 0, 1]
    mz1_right=array[:, int((5/8*l)/cx), int(w/2/cy), 0, 2]
    my1_right=array[:, int((5/8*l)/cx), int(w/2/cy), 0, 1]
    T1z=np.max(mz1_right)**2/np.max(mz1_left)**2
    T1y=np.max(my1_right)**2/np.max(my1_left)**2
    my_BL, mz_BL=baseline
    T2z=np.max(mz1_right)**2/mz_BL**2
    T2y=np.max(my1_right)**2/my_BL**2
    return T1z, T1y, T2z, T2y   

In [4]:
with open(f"./Transmissions.csv",'a') as f:
    f.write(f"Transverse field,Frequency,T1z,T1y,T2z,T2y\n")

for freq in f_MAX_list:    
    for Hy_t in Hy_list:
        
        sysName=f"P3_{int(T*1e9)}ns_{int(freq*1e-6)}kHz_{int(Hy_t/Oe)}Oe"    

        if Hy_t==0:
            data=md.Data(sysName) #this contains all the drives up to now. [-1] means the last drive. Check the folder "Py_disk_FMR"
            array=data[-1].to_xarray()
            mz_BL=np.max(array[:, int((3/8*l)/cx), int(w/2/cy), 0, 2])
            my_BL=np.max(array[:, int((3/8*l)/cx), int(w/2/cy), 0, 1])

        a,b,c,d=getTramissions((my_BL,mz_BL))
        a = float(a)
        b = float(b)
        c = float(c)
        d = float(d)


        with open(f"./Transmissions.csv",'a') as f:
            f.write(f"{Hy_t},{freq},{a},{b},{c},{d}\n")
            # f.write(f"Transverse field=:{Hy_t}\n")
            # f.write(f"frequency={freq}\n")
            # f.write(f"T1z={a}\n")
            # f.write(f"T1y={b}\n")
            # f.write(f"T2z={c}\n")
            # f.write(f"T2y={d}\n")
            # f.write("\n\n")
            